# Tidying Up Your Data
Process of preparing raw data for analysis

Tidy data is a term that was created in what many refer to as a famous data science
paper, "Tidy Data" by Hadley Wickham, which I highly recommend that you read
and it can be downloaded at http://vita.had.co.nz/papers/tidy-data.pdf. 

There're several characteristics of data that can be considered ready for analysis:
* Each variable is one column.
* Each observation of the variable is in a different row.
* There should be one table for each kind of variable.
* If multiple tables, they should be relatable.
* Qualitative and categorical variables have mappings to values useful for analysis.

In [1]:
# import pandas, numpy and datetime
import numpy as np
import pandas as pd
import datetime

# Set some pandas options for controlling output
pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', 10)

## Working with missing data
In pandas, there are a number of reasons why a value can be NaN:
* A join of two sets of data does not have matched values
* Data that you retrieved from an external source is incomplete
* The NaN value is not known at a given point in time and will be filled in later
* There is a data collection error retrieving a value, but the event must still be recorded in the index
* Reindexing of data has resulted in an index that does not have a value
* The shape of data has changed and there are now additional rows or columns, which at the time of reshaping could not be determined

In [3]:
# create a DataFrame with 5 rows and 3 columns
df = pd.DataFrame(np.arange(0, 15).reshape(5, 3),
 index=['a', 'b', 'c', 'd', 'e'],
 columns=['c1', 'c2', 'c3'])

# add some columns and rows to the DataFrame
df['c4'] = np.nan
df.loc['f'] = np.arange(15, 19)
df.loc['g'] = np.nan
df['c5'] = np.nan
df['c4']['a'] = 20

df

     c1    c2    c3    c4  c5
a   0.0   1.0   2.0  20.0 NaN
b   3.0   4.0   5.0   NaN NaN
c   6.0   7.0   8.0   NaN NaN
d   9.0  10.0  11.0   NaN NaN
e  12.0  13.0  14.0   NaN NaN
f  15.0  16.0  17.0  18.0 NaN
g   NaN   NaN   NaN   NaN NaN

### Determining NaN values in Series and DataFrame objects

In [4]:
df.isnull()

      c1     c2     c3     c4    c5
a  False  False  False  False  True
b  False  False  False   True  True
c  False  False  False   True  True
d  False  False  False   True  True
e  False  False  False   True  True
f  False  False  False  False  True
g   True   True   True   True  True

In [5]:
df.isnull().sum()

c1    1
c2    1
c3    1
c4    5
c5    7
dtype: int64

In [6]:
df.isnull().sum().sum()

15

In [7]:
# another method is using count() since it counts non NAN values only
(len(df) - df.count()).sum()

15

In [8]:
# which items are not null?
df.notnull()

      c1     c2     c3     c4     c5
a   True   True   True   True  False
b   True   True   True  False  False
c   True   True   True  False  False
d   True   True   True  False  False
e   True   True   True  False  False
f   True   True   True   True  False
g  False  False  False  False  False

### Selecting out or dropping missing data

In [10]:
# select the non-NaN items in column c4
df.c4[df.c4.notnull()]

a    20.0
f    18.0
Name: c4, dtype: float64

In [12]:
# .dropna will also return non NaN values
# this gets all non NaN items in column c4
# it returns a copy of the DataFrame
df.c4.dropna()

a    20.0
f    18.0
Name: c4, dtype: float64

In [13]:
# applying .dropna to a DataFram it drops all rows from a DataFrame object that have at least one NaN value
df.dropna()


Empty DataFrame
Columns: [c1, c2, c3, c4, c5]
Index: []

In [14]:
# using how='all', only rows that have all values
# as NaN will be dropped
df.dropna(how = 'all')

     c1    c2    c3    c4  c5
a   0.0   1.0   2.0  20.0 NaN
b   3.0   4.0   5.0   NaN NaN
c   6.0   7.0   8.0   NaN NaN
d   9.0  10.0  11.0   NaN NaN
e  12.0  13.0  14.0   NaN NaN
f  15.0  16.0  17.0  18.0 NaN

In [15]:
# flip to drop columns instead of rows
df.dropna(how='all', axis=1) # say goodbye to c5

# df.dropna(how='any', axis=1) # drop columns with any NaN values
# df.dropna(thresh=5, axis=1) # only drop columns with at least 5 NaN values
# notice that it returns a copy.  If you want to drop the data in the actual DataFrame, 
# use the inplace=True parameter.

     c1    c2    c3    c4
a   0.0   1.0   2.0  20.0
b   3.0   4.0   5.0   NaN
c   6.0   7.0   8.0   NaN
d   9.0  10.0  11.0   NaN
e  12.0  13.0  14.0   NaN
f  15.0  16.0  17.0  18.0
g   NaN   NaN   NaN   NaN

### How pandas handles NaN values in mathematical operations
NumPy functions, when encountering a NaN value, will return NaN. pandas functions
and will typically ignore the NaN values and continue processing the function as
though the values were not part of the Series object.

In [21]:
# demonstrate sum, mean and cumsum handling of NaN
# get one column
s = df.c4
s.sum() # NaN values treated as 0
s.mean() # NaN also treated as 0
s.cumsum() # as 0 in the cumsum, but NaN values preserved in result Series
df.c4 + 1 # in arithmetic, a NaN value will result in NaN

a    21.0
b     NaN
c     NaN
d     NaN
e     NaN
f    19.0
g     NaN
Name: c4, dtype: float64

### Filling in missing data

In [22]:
# return a new DataFrame with NaN values filled with 0
filled = df.fillna(0)
filled

     c1    c2    c3    c4   c5
a   0.0   1.0   2.0  20.0  0.0
b   3.0   4.0   5.0   0.0  0.0
c   6.0   7.0   8.0   0.0  0.0
d   9.0  10.0  11.0   0.0  0.0
e  12.0  13.0  14.0   0.0  0.0
f  15.0  16.0  17.0  18.0  0.0
g   0.0   0.0   0.0   0.0  0.0

In [23]:
# the following code shows the result of applying the .mean() method to the DataFrame
# object with the NaN values, as compared to the DataFrame that has its NaN values
# filled with 0:
# NaNs don't count as an item in calculating

df.mean()

c1     7.5
c2     8.5
c3     9.5
c4    19.0
c5     NaN
dtype: float64

In [24]:
# having replaced NaN with 0 can make
# operations such as mean have different results
filled.mean()

c1    6.428571
c2    7.285714
c3    8.142857
c4    5.428571
c5    0.000000
dtype: float64

In [25]:
# only fills the first two NaN values in each row with 0
df.fillna(0, limit=2)

     c1    c2    c3    c4   c5
a   0.0   1.0   2.0  20.0  0.0
b   3.0   4.0   5.0   0.0  0.0
c   6.0   7.0   8.0   0.0  NaN
d   9.0  10.0  11.0   NaN  NaN
e  12.0  13.0  14.0   NaN  NaN
f  15.0  16.0  17.0  18.0  NaN
g   0.0   0.0   0.0   NaN  NaN

### Forward and backward filling of missing values

In [27]:
# extract the c4 column and fill NaNs forward
df.c4.fillna(method="ffill")

a    20.0
b    20.0
c    20.0
d    20.0
e    20.0
f    18.0
g    18.0
Name: c4, dtype: float64

In [28]:
# perform a backwards fill
df.c4.fillna(method="bfill")

a    20.0
b    18.0
c    18.0
d    18.0
e    18.0
f    18.0
g     NaN
Name: c4, dtype: float64

### Filling using index labels

In [29]:
# create a new Series of values to be
# used to fill NaN values where the index label matches
fill_values = pd.Series([100, 101, 102], index=['a', 'e', 'g'])

# using c4, fill using fill_values
# a, e and g will be filled with matching values
df.c4.fillna(fill_values)

a     20.0
b      NaN
c      NaN
d      NaN
e    101.0
f     18.0
g    102.0
Name: c4, dtype: float64

### Filling with the mean

In [30]:
# fill NaN values in each column with the
# mean of the values in that column
df.fillna(df.mean())

     c1    c2    c3    c4  c5
a   0.0   1.0   2.0  20.0 NaN
b   3.0   4.0   5.0  19.0 NaN
c   6.0   7.0   8.0  19.0 NaN
d   9.0  10.0  11.0  19.0 NaN
e  12.0  13.0  14.0  19.0 NaN
f  15.0  16.0  17.0  18.0 NaN
g   7.5   8.5   9.5  19.0 NaN

### Interpolation of missing values

In [32]:
# linear interpolate the NaN values from 1 through 2
s = pd.Series([1, np.nan, np.nan, np.nan, 2])
s.interpolate()

0    1.00
1    1.25
2    1.50
3    1.75
4    2.00
dtype: float64

Notice:  (2.0 – 1.0)/(5-1) = 0.25

In [34]:
# create a time series, but missing one date in the Series
ts = pd.Series([1, np.nan, 2],
index=[datetime.datetime(2014, 1, 1),
datetime.datetime(2014, 2, 1),
datetime.datetime(2014, 4, 1)])
ts

2014-01-01    1.0
2014-02-01    NaN
2014-04-01    2.0
dtype: float64

In [35]:
# linear interpolate based on the number of items in the Series
ts.interpolate()


2014-01-01    1.0
2014-02-01    1.5
2014-04-01    2.0
dtype: float64

In [36]:
# this accounts for the fact that we don't have
# an entry for 2014-03-01
ts.interpolate(method="time")

2014-01-01    1.000000
2014-02-01    1.344444
2014-04-01    2.000000
dtype: float64

In [37]:
# a Series to demonstrate index label based interpolation
s = pd.Series([0, np.nan, 100], index=[0, 1, 10])
s

0       0.0
1       NaN
10    100.0
dtype: float64

In [38]:
# linear interpolate
s.interpolate()

0       0.0
1      50.0
10    100.0
dtype: float64

In [39]:
# interpolate based upon the values in the index
s.interpolate(method="values")

0       0.0
1      10.0
10    100.0
dtype: float64

### Handling duplicate data

In [41]:
# a DataFrame with lots of duplicate data
data = pd.DataFrame({'a': ['x'] * 3 + ['y'] * 4,
 'b': [1, 1, 2, 3, 3, 4, 4]})
data

   a  b
0  x  1
1  x  1
2  x  2
3  y  3
4  y  3
5  y  4
6  y  4

In [42]:
# reports which rows are duplicates based upon
# if the data in all columns was seen before
data.duplicated()

0    False
1     True
2    False
3    False
4     True
5    False
6     True
dtype: bool

In [44]:
# return a copy of the DataFrame with the duplicates dropped
# you can use inplace=True
data.drop_duplicates()


   a  b
0  x  1
2  x  2
3  y  3
5  y  4

In [49]:
# if you want to keep the last instance ( the default is the first )
data.drop_duplicates(keep='last')

   a  b
1  x  1
2  x  2
4  y  3
6  y  4

In [51]:
# add a column c with values 0..6
# this makes .duplicated() report no duplicate rows
data['c'] = range(7)
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6    False
dtype: bool

In [52]:
# if we specify duplicates to be dropped only in columns a & b
# they will be dropped
data.drop_duplicates(['a', 'b'])

   a  b  c
0  x  1  0
2  x  2  2
3  y  3  3
5  y  4  5

## Transforming Data
transforming existing data into another presentation. This may be needed for the following reasons:
* Values are not in the correct units
* Values are qualitative and need to be converted to appropriate numeric values
* There is extraneous data that either wastes memory and processing time, or can affect results simply by being included

To address these situations, we can take one or more of the following actions:
* Map values to other values using a table lookup process
* Explicitly replace certain values with other values (or even another type of data)
* Apply methods to transform the values based on an algorithm
* Simply remove extraneous columns and rows

### Mapping

In [54]:
# You can use a dictionary or a pandas Series

# create two Series objects to demonstrate mapping
x = pd.Series({"one": 1, "two": 2, "three": 3})
y = pd.Series({1: "a", 2: "b", 3: "c"})

# map values in x to values in y
x.map(y)

one      a
two      b
three    c
dtype: object

In [55]:
# three in x will not align / map to a value in y
x = pd.Series({"one": 1, "two": 2, "three": 3})
y = pd.Series({1: "a", 2: "b"})
x.map(y)

one        a
two        b
three    NaN
dtype: object

### Replacing values
We previously saw how the .fillna() method can be used to replace the NaN
values with a value of your own decision. 

The .fillna() method can actually be
thought of as an implementation of the .map() method that maps a single value,
NaN, to a specific value.


Even more generically, the .fillna() method itself can be considered a
specialization of a more general replacement that is provided by the .replace()

In [56]:
# create a Series to demonstrate replace
s = pd.Series([0., 1., 2., 3., 2., 4.])
s

0    0.0
1    1.0
2    2.0
3    3.0
4    2.0
5    4.0
dtype: float64

In [57]:
# replace all items with index label 2 with value 5
s.replace(2, 5)

# replace all items with new values
# s.replace([0, 1, 2, 3, 4], [4, 3, 2, 1, 0])

# replace using entries in a dictionary
# s.replace({0: 10, 1: 100})

0    0.0
1    1.0
2    5.0
3    3.0
4    5.0
5    4.0
dtype: float64

In [58]:
# DataFrame with two columns
df = pd.DataFrame({'a': [0, 1, 2, 3, 4], 'b': [5, 6, 7, 8, 9]})
df

   a  b
0  0  5
1  1  6
2  2  7
3  3  8
4  4  9

In [59]:
# specify different replacement values for each column
df.replace({'a': 1, 'b': 8}, 100)

     a    b
0    0    5
1  100    6
2    2    7
3    3  100
4    4    9

### Applying functions to transform data


In [60]:
# demonstrate applying a function to every item of a Series
s = pd.Series(np.arange(0, 5))
s.apply(lambda v: v * 2)

0    0
1    2
2    4
3    6
4    8
dtype: int64

Notice: When applying a function to items in a Series, only the value for each Series item is
passed to the function, not the index label and the value.

When a function is applied to a DataFrame. pandas will iterate through all columns passing each as a Series to
your function. The result will be a Series object with index labels matching column
names and with the result of the function applied to the column:

In [62]:
# demonstrate applying a sum on each column
df = pd.DataFrame(np.arange(12).reshape(4, 3),
columns=['a', 'b', 'c'])
df

   a   b   c
0  0   1   2
1  3   4   5
2  6   7   8
3  9  10  11

In [63]:
df.apply(lambda col: col.sum())

a    18
b    22
c    26
dtype: int64

In [64]:
# calculate the sum of items in each row
df.apply(lambda row: row.sum(), axis=1)


0     3
1    12
2    21
3    30
dtype: int64

In [65]:
# create a new column 'interim' with a * b
df['interim'] = df.apply(lambda r: r.a * r.b, axis=1)
df

   a   b   c  interim
0  0   1   2        0
1  3   4   5       12
2  6   7   8       42
3  9  10  11       90

The .apply() method will always apply to the provided function to all of the items,
or rows or columns. If you want to apply the function to a subset of these, then first
perform a Boolean selection to filter the items you do not want process.

In [66]:
# create a 3x5 DataFrame
# only second row has a NaN
df = pd.DataFrame(np.arange(0, 15).reshape(3,5))
df.loc[1, 2] = np.nan
df

    0   1     2   3   4
0   0   1   2.0   3   4
1   5   6   NaN   8   9
2  10  11  12.0  13  14

In [67]:
# demonstrate applying a function to only rows having
# a count of 0 NaN values
df.dropna().apply(lambda x: x.sum(), axis=1)

    0   1     2   3   4
0   0   1   2.0   3   4
2  10  11  12.0  13  14

The .apply() method was
always passed an entire row or column. If you desire to apply a function to every
individual item in the DataFrame one by one, then .applymap() is the method to use.

In [68]:
# use applymap to format all items of the DataFrame
df.applymap(lambda x: '%.2f' % x)

       0      1      2      3      4
0   0.00   1.00   2.00   3.00   4.00
1   5.00   6.00    nan   8.00   9.00
2  10.00  11.00  12.00  13.00  14.00